## Raviraj_Wagh
___Cointab_DA-Challenge_Output___

###### Import files

In [1]:
import pandas as pd
file_1=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - Order Report.xlsx")
file_2=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - Pincode Zones.xlsx")
file_3=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - SKU Master.xlsx")
file_4=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Courier Company - Invoice.xlsx")
file_5=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Courier Company - Rates.xlsx")
file_out_1=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Expected_Result.xlsx",sheet_name='Summary')
file_out_2=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Expected_Result.xlsx",sheet_name='Calculations')

In [2]:
file_1['ExternOrderNo'].nunique()

124

##### Handling Duplicates

In [3]:
file_1.drop_duplicates(inplace=True)
file_2.drop_duplicates(inplace=True)
file_3.drop_duplicates(inplace=True)

###### Merging Order quantity and Weight as pe SKU

In [4]:
new_1=file_1.merge(file_3,how='left',on='SKU')

In [5]:
# conversion of weight grms to kg
new_1['Weight (kg)']=new_1['Weight (g)']/1000
new_1.drop('Weight (g)',inplace=True,axis=1)

In [6]:
new_1['tot']=new_1['Order Qty']*new_1['Weight (kg)']

###### Unique order ID's by Grouping them

In [7]:
new_2=new_1.groupby('ExternOrderNo').sum().reset_index()
new_2=new_2.drop('Order Qty',axis=1)
new_2=new_2.drop('Weight (kg)',axis=1)

In [8]:
new_2.rename(columns={'ExternOrderNo':'Order ID'},inplace=True)

###### Merging AWB code

In [9]:
new_3=new_2.merge(file_4[['AWB Code','Order ID']],how='left',on='Order ID')

###### Building Slab for x

In [10]:
a=new_3['tot']//1
b=new_3['tot']%1

b[(b>0)&(b<0.5)]=0.5
b[b>0.5]=1

new_3['slab_wt']=a+b

In [11]:
new_31=new_3.merge(file_4[['Order ID','Charged Weight','Customer Pincode','Zone','Type of Shipment','Billing Amount (Rs.)']],on='Order ID')
new_31.rename(columns={'Zone':'Zone as per courier'},inplace=True)

In [12]:
new_4=new_31.merge(file_2[['Customer Pincode','Zone']],on='Customer Pincode')
new_4.rename(columns={'Zone':'Zone as per X'},inplace=True)

###### Buiding Slab for Courier Company

In [13]:
a=new_4['Charged Weight']//1
b=new_4['Charged Weight']%1

b[(b>0)&(b<0.5)]=0.5
b[b>0.5]=1

new_4['slab_wt_As_per_Courier_company']=a+b

In [14]:
new_4[new_4['Type of Shipment']=='Forward charges']['Zone as per X'].unique()

array(['b', 'd', 'e'], dtype=object)

In [15]:
new_4[new_4['Type of Shipment']=='Forward and RTO charges']['Zone as per X'].unique()

array(['d', 'e', 'b'], dtype=object)

###### Expected_charges_as_per_X

In [16]:
new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='b') & (new_4['slab_wt']<=0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_b_fixed']
new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='b') & (new_4['slab_wt']>0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_b_fixed']+(new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='b') & (new_4['slab_wt']>0.5)]['slab_wt']/0.5-1)*file_5.loc[0,'fwd_b_additional']
new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='d') & (new_4['slab_wt']<=0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_d_fixed']
new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='d') & (new_4['slab_wt']>0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_d_fixed']+(new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='d') & (new_4['slab_wt']>0.5)]['slab_wt']/0.5-1)*file_5.loc[0,'fwd_d_additional']
new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='e') & (new_4['slab_wt']<=0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_e_fixed']
new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='e') & (new_4['slab_wt']>0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_e_fixed']+(new_4.loc[(new_4['Type of Shipment']=='Forward charges') & (new_4['Zone as per X']=='e') & (new_4['slab_wt']>0.5)]['slab_wt']/0.5-1)*file_5.loc[0,'fwd_e_additional']


new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='b') & (new_4['slab_wt']<=0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_b_fixed']+file_5.loc[0,'rto_b_fixed']
new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='b') & (new_4['slab_wt']>0.5),'Expected_charges_as_per_X']=(file_5.loc[0,'fwd_b_fixed']+file_5.loc[0,'rto_b_fixed'])+(new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='b') & (new_4['slab_wt']>0.5)]['slab_wt']/0.5-1)*(file_5.loc[0,'fwd_b_additional']+file_5.loc[0,'rto_b_additional'])
new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='d') & (new_4['slab_wt']<=0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_d_fixed']+file_5.loc[0,'rto_d_fixed']
new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='d') & (new_4['slab_wt']>0.5),'Expected_charges_as_per_X']=(file_5.loc[0,'fwd_d_fixed']+file_5.loc[0,'rto_d_fixed'])+(new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='d') & (new_4['slab_wt']>0.5)]['slab_wt']/0.5-1)*(file_5.loc[0,'fwd_d_additional']+file_5.loc[0,'rto_d_additional'])
new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='e') & (new_4['slab_wt']<=0.5),'Expected_charges_as_per_X']=file_5.loc[0,'fwd_e_fixed']+file_5.loc[0,'rto_e_fixed']
new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='e') & (new_4['slab_wt']>0.5),'Expected_charges_as_per_X']=(file_5.loc[0,'fwd_e_fixed']+file_5.loc[0,'rto_e_fixed'])+(new_4.loc[(new_4['Type of Shipment']=='Forward and RTO charges') & (new_4['Zone as per X']=='e') & (new_4['slab_wt']>0.5)]['slab_wt']/0.5-1)*(file_5.loc[0,'fwd_e_additional']+file_5.loc[0,'rto_e_additional'])



In [17]:
seq=['Order ID', 'AWB Code','tot', 'slab_wt', 'Charged Weight','slab_wt_As_per_Courier_company',
         'Zone as per X','Zone as per courier','Expected_charges_as_per_X', 'Billing Amount (Rs.)']
seq

['Order ID',
 'AWB Code',
 'tot',
 'slab_wt',
 'Charged Weight',
 'slab_wt_As_per_Courier_company',
 'Zone as per X',
 'Zone as per courier',
 'Expected_charges_as_per_X',
 'Billing Amount (Rs.)']

In [18]:
new_4 = new_4[seq]

In [19]:
new_4=new_4.rename(columns={'Billing Amount (Rs.)':'Billing_Amount_(Rs.)'})

In [20]:
new_4['Diffs']=new_4['Expected_charges_as_per_X']-new_4['Billing_Amount_(Rs.)']

In [21]:
labels=['Order_ID', 'AWB_Number', 'Total_weight_as_per_X_(KG)',
       'Weight_slab_as_per_X_(KG)', 'Total_weight_as_per_Courier_Company_(KG)',
       'Weight_sla_b_charged_by_Courier_Company_(KG)', 'Delivery_Zone_as_per_X',
       'Delivery_Zone_charged_by_Courier_Company',
       'Expected_Charge_as_per_X_(Rs.)',
       'Charges_Billed_by_Courier_Company_(Rs.) ',
       'Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']

In [22]:
for i,j in zip(new_4.columns,labels):
    new_4=new_4.rename(columns={i:j},inplace=False)

In [23]:
file_out_2=new_4
file_out_2

,Order_ID,AWB_Number,Total_weight_as_per_X_(KG),Weight_slab_as_per_X_(KG),Total_weight_as_per_Courier_Company_(KG),Weight_sla_b_charged_by_Courier_Company_(KG),Delivery_Zone_as_per_X,Delivery_Zone_charged_by_Courier_Company,Expected_Charge_as_per_X_(Rs.),Charges_Billed_by_Courier_Company_(Rs.),Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,b,d,33.0,45.4,-12.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,b,d,89.6,135.0,-45.4
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,d,d,176.3,172.8,3.5
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,d,d,355.5,345.0,10.5


### summary table

In [24]:
file_out_1=file_out_1.rename(columns={'Unnamed: 0':'Types_of_charges'})

In [25]:
file_out_1.at[0,'Count']=file_out_2.loc[file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']==0,'Charges_Billed_by_Courier_Company_(Rs.) '].count()
file_out_1.at[0,'Amount']=file_out_2['Charges_Billed_by_Courier_Company_(Rs.) '][file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']==0].sum()

file_out_1.at[1,'Count']=file_out_2.loc[file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']<0,'Charges_Billed_by_Courier_Company_(Rs.) '].count()
file_out_1.at[1,'Amount']=file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)'][file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']<0].abs().sum()

file_out_1.at[2,'Count']=file_out_2.loc[file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']>0,'Charges_Billed_by_Courier_Company_(Rs.) '].count()
file_out_1.at[2,'Amount']=file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)'][file_out_2['Difference_Between_Expected_Charges_and_Billed_Charges_(Rs.)']>0].abs().sum()


In [26]:
file_out_1

,Types_of_charges,Count,Amount
0,Total Orders - Correctly Charged,15.0,1207.3
1,Total Orders - Over Charged,85.0,4483.2
2,Total Orders - Under Charged,24.0,575.1


# Exporting files in Exel work Books

In [27]:
"""
import xlsxwriter

excel_writer = pd.ExcelWriter(r'C:\Users\Parimal\Desktop\Cointab_DA-Challenge_Output_(Raviraj_Wagh).xlsx', engine='xlsxwriter')
file_out_1.to_excel(excel_writer, sheet_name='Summary_table',index=False)
file_out_2.to_excel(excel_writer, sheet_name='order_level_calculation',index=False)
excel_writer.save()
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 54-55: truncated \UXXXXXXXX escape (Temp/ipykernel_18812/4103837602.py, line 8)

In [ ]:
joaiojif